The dataset is very large - 235mb so long time to upload the dataset onto kaggle so we will direct the data from kaggle to colab using an api

In [1]:
!pip install kaggle

In [1]:
#configure the path of kaggle json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
#API to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140
#api link copied from kaggle

sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
#extracting content from zip file
from zipfile import ZipFile
df = '/content/sentiment140.zip'
with ZipFile(df,'r') as zip: #reading the dataset file
  zip.extractall() #extracting the contents
  print('Extraction of data is successfully completed')

Extraction of data is successfully completed


In [9]:
import pandas as pd
import numpy as np
import re #regular expression
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [10]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
#data preprocessing
td = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1')

In [13]:
td.shape
td.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


above column names are not being read and the first data is being considered as column names

In [14]:
#name columns and read the data again
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
td = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names = column_names, encoding='ISO-8859-1')

In [15]:
td.shape
td.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [17]:
td.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [18]:
#distribution
td['target'].value_counts()
#number of 0-neg and 4-pos tweets

0    800000
4    800000
Name: target, dtype: int64

the dataset is even but if the dataset doesn't have even distribution then we have to work with up or down sampling else the model won't work properly

In [19]:
#convert 4 to 1 as 1 - pos want for processing in ml model
td.replace({'target':{4:1}}, inplace=True)

In [21]:
#Stem the words
portstem = PorterStemmer()

In [25]:
def stemming(content):
  stem_content = re.sub('[^a-zA-Z]', ' ', content) #remove all the words that are not in a-z or A-Z - symbols and punctuations
  # ^ = for removing
  stem_content = stem_content.lower()
  stem_content = stem_content.split()
  stem_content = [portstem.stem(word) for word in stem_content if not word in stopwords.words('english')]
  stem_content = ' '.join(stem_content)
  return stem_content

In [27]:
td['stem_content'] = td['text'].apply(stemming);

KeyboardInterrupt: 